In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy import stats

import ecephys_analyses as ea 
from ecephys.signal.xarray_utils import filter_dataset_by_hypnogram

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]
experiment = "sleep-homeostasis"
norm_condition = "light-period-circadian-match"
bsl_condition = "recovery-sleep-circadian-match"
rec_condition = "recovery-sleep"
states = ["N1", "N2"]
duration = '02:00:00'
light_period_start = '09:00:00'
light_period_end = '21:00:00'
f_range = (0.5, 4)

In [24]:
def get_subject_data(
    subject,
    experiment,
    norm_condition,
    bsl_condition,
    rec_condition,
    states,
    duration,
    f_range,
    light_period_start,
    light_period_end,
):

    norm_hyp = ea.load_hypnogram(subject, experiment, norm_condition)
    norm_hyp = norm_hyp.keep_states(states).keep_between(
        light_period_start, light_period_end
    )

    norm_pow = (
        ea.load_power(subject, experiment, norm_condition, "spg.nc")
        .sel(frequency=slice(*f_range))
        .sum(dim="frequency")
    )
    norm_pow = (
        filter_dataset_by_hypnogram(norm_pow, norm_hyp)
        .swap_dims({"time": "datetime"})
        .drop_vars(["time", "timedelta"])
    )

    rec_hyp = ea.load_hypnogram(subject, experiment, rec_condition)
    rec_hyp = rec_hyp.keep_states(states).keep_first(duration)

    rec_pow = (
        ea.load_power(subject, experiment, rec_condition, "spg.nc")
        .sel(frequency=slice(*f_range))
        .sum(dim="frequency")
    )
    rec_pow = (
        filter_dataset_by_hypnogram(rec_pow, rec_hyp)
        .swap_dims({"time": "datetime"})
        .drop_vars(["time", "timedelta"])
    )
    rec_pow = rec_pow / norm_pow.mean(dim="datetime") * 100

    bsl_hyp = ea.load_hypnogram(subject, experiment, bsl_condition)
    bsl_hyp = (
        bsl_hyp.keep_states(states)
        .keep_between(rec_hyp.start_time.min().strftime("%H:%M:%S"), light_period_end)
        .keep_first(duration)
    )

    bsl_pow = (
        ea.load_power(subject, experiment, bsl_condition, "spg.nc")
        .sel(frequency=slice(*f_range))
        .sum(dim="frequency")
    )
    bsl_pow = (
        filter_dataset_by_hypnogram(bsl_pow, bsl_hyp)
        .swap_dims({"time": "datetime"})
        .drop_vars(["time", "timedelta"])
    )
    bsl_pow = bsl_pow / norm_pow.mean(dim="datetime") * 100

    diff_means = rec_pow.mean() - bsl_pow.mean()
    diff_meds = rec_pow.median() - bsl_pow.median()

    index = pd.MultiIndex.from_product(
        [[subject], ["cortex", "hippocampus"]], names=["subject", "region"]
    )
    data = np.asarray(
        [
            [
                diff_means.mpta_wm_ref,
                diff_meds.mpta_wm_ref,
            ],
            [
                diff_means.sr_wm_ref,
                diff_meds.sr_wm_ref,
            ],
        ]
    )
    return pd.DataFrame(
        data,
        columns=["diff_means", "diff_meds"],
        index=index,
    )

In [25]:
subj_data = list()
for subject in subjects:
    subj_data.append(get_subject_data(subject, experiment, norm_condition, bsl_condition, rec_condition, states, duration, f_range, light_period_start, light_period_end))
df = pd.concat(subj_data)

In [26]:
print(df.to_markdown())

|                               |   diff_means |   diff_meds |
|:------------------------------|-------------:|------------:|
| ('Segundo', 'cortex')         |     174.666  |    175.633  |
| ('Segundo', 'hippocampus')    |      42.015  |     46.0376 |
| ('Valentino', 'cortex')       |     103.253  |     85.2942 |
| ('Valentino', 'hippocampus')  |      35.9414 |     39.6776 |
| ('Doppio', 'cortex')          |     157.878  |    156.224  |
| ('Doppio', 'hippocampus')     |      43.6544 |     40.87   |
| ('Alessandro', 'cortex')      |     108.873  |    124.296  |
| ('Alessandro', 'hippocampus') |      40.0028 |     52.5756 |
| ('Allan', 'cortex')           |     189.006  |    190.341  |
| ('Allan', 'hippocampus')      |      33.7885 |     31.3412 |


In [27]:
cx = df.xs('cortex', level=1, drop_level=False)
hc = df.xs('hippocampus', level=1, drop_level=False)

In [28]:
stats.ttest_1samp(cx, 0)

Ttest_1sampResult(statistic=array([8.46101728, 7.76543915]), pvalue=array([0.00106921, 0.0014823 ]))

In [29]:
stats.ttest_1samp(hc, 0)

Ttest_1sampResult(statistic=array([21.15670024, 11.94664386]), pvalue=array([2.95065648e-05, 2.81286504e-04]))